In [1]:
import torch
import torch.nn as nn # Make biases and weights part of the network
import torch.nn.functional as F # Activation Function
from torch.optim import Adam # SGD 

import lightning as L 
from torch.utils.data import TensorDataset, DataLoader

In [2]:
class LSTMbyHand(L.LightningModule):
    def __init__(self):
        # Create and initialize weights and biases tensors
        super().__init__()
        # Normal Distribution to generate random number for weights
        
        self.lstm = nn.LSTM(input_size=1, hidden_size=3) 
        # input size = number of variable in our training data
        # hidden size = number of values that we want - in the example we just need day 5
            # 
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)
        
        self.wlr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wlr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.blr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)
        
        self.wpr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wpr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bpr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)
        
        self.wp1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wp2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bp1 = nn.Parameter(torch.tensor(0.), requires_grad=True)
        
        self.wo1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wo2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bo1 = nn.Parameter(torch.tensor(0.), requires_grad=True)
        
        
    def lstm_unit(self, input_value, long_memory, short_memory):
        long_remember_percent = torch.sigmoid((short_memory * self.wlr1) +
                                              (input_value * self.wlr2) +
                                              self.blr1)
        
        potential_remember_percent = torch.sigmoid((short_memory * self.wpr1) +
                                                  (input_value * self.wpr2) +
                                                  self.bpr1)
        
        potential_memory = torch.tanh((short_memory * self.wp1) + 
                                     (input_value * self.wp2) +
                                     self.bp1)
        
        updated_long_memory = ((long_memory * long_remember_percent) + 
                              (potential_remember_percent * potential_memory))
        
        output_percent = torch.sigmoid((short_memory * self.wo1) + 
                                      (input_value * self.wo2) +
                                      self.bo1)
        
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent
        
        return([updated_long_memory, updated_short_memory])
    
    
    def forward(self, input):
        input_trans = input.view(len(input), 1) 
        # Transpose input the values from one the companies for days through 4 from being in a single row 
        # to being in a single columns with view()
        
        # Input start by inputting how many rows we have in this 
        lstm_out, temp = self.lstm(input_trans)
        
        prediction = lstm_out[-1]
        return prediction
    
        ###
#         long_memory = 0
#         short_memory = 0
#         day1 = input[0]
#         day2 = input[1]
#         day3 = input[2]
#         day4 = input[3]
        
#         long_memory, short_memory = self.lstm_unit(day1, long_memory, short_memory)
#         long_memory, short_memory = self.lstm_unit(day2, long_memory, short_memory)
#         long_memory, short_memory = self.lstm_unit(day3, long_memory, short_memory)
#         long_memory, short_memory = self.lstm_unit(day4, long_memory, short_memory)
    
#         return short_memory
    
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)
    
    
    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i) ** 2
        
        self.log("Train_loss", loss)
        
        if (label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)
        
        return loss
    

In [3]:
model = LSTMbyHand()

print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0,  Predicted =",
     model(torch.tensor([0., 0.5, 0.25, 1.])).detach())

print("Company B: Observed = 1, Predicted =",
     model(torch.tensor([1., 0.5, 0.25, 1.])).detach())



Now let's compare the observed and predicted values...
Company A: Observed = 0,  Predicted = tensor([-0.0555, -0.0389, -0.2769])
Company B: Observed = 1, Predicted = tensor([-0.0221, -0.0262, -0.2786])


In [10]:
trainer = L.Trainer(max_epochs=300, log_every_n_steps=2)
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path
trainer.fit(model, train_dataloaders=DataLoader, ckpt_path=path_to_best_checkpoint)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


ValueError: `.fit()` found no path for the best weights: ''. Please specify a path for a checkpoint `.fit(ckpt_path=PATH)`

In [5]:
trainer = Trainer()

NameError: name 'Trainer' is not defined

In [6]:
import math
print(dir(math))

['__doc__', '__loader__', '__name__', '__package__', '__spec__', 'acos', 'acosh', 'asin', 'asinh', 'atan', 'atan2', 'atanh', 'ceil', 'comb', 'copysign', 'cos', 'cosh', 'degrees', 'dist', 'e', 'erf', 'erfc', 'exp', 'expm1', 'fabs', 'factorial', 'floor', 'fmod', 'frexp', 'fsum', 'gamma', 'gcd', 'hypot', 'inf', 'isclose', 'isfinite', 'isinf', 'isnan', 'isqrt', 'lcm', 'ldexp', 'lgamma', 'log', 'log10', 'log1p', 'log2', 'modf', 'nan', 'nextafter', 'perm', 'pi', 'pow', 'prod', 'radians', 'remainder', 'sin', 'sinh', 'sqrt', 'tan', 'tanh', 'tau', 'trunc', 'ulp']


In [6]:
print(dir(L))
print(L.Trainer())

['BuildConfig', 'Callback', 'CloudCompute', 'Fabric', 'LightningApp', 'LightningDataModule', 'LightningFlow', 'LightningModule', 'LightningWork', 'Trainer', '__about__', '__all__', '__author__', '__author_email__', '__builtins__', '__cached__', '__copyright__', '__doc__', '__docs__', '__docs_url__', '__file__', '__homepage__', '__license__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_console', '_logger', 'app', 'fabric', 'formatter', 'lightning', 'logging', 'os', 'pdb', 'pytorch', 'seed_everything', 'storage']


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
L.__version__

'2.0.2'